# Проект

### оставь надежду, всяк сюда входящий - возьми лучше корпус

ячейка для импортов

In [5]:
%pip install pymorphy2
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from nltk import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

import pandas as pd

import numpy as np

ua = UserAgent(verify_ssl=False)
session = requests.session()

     |████████████████████████████████| 61kB 2.7MB/s 
     |████████████████████████████████| 8.2MB 5.7MB/s 


Ячейка для обкачивания ссылок на новости с сайта гордой газеты Московский комсомолец, а точнее, ее раздела "проишествия"

In [6]:
news = []
page_number = 0
for i in tqdm(range(4)):
    page_number += 1
    url = f'https://www.mk.ru/incident/{page_number}/'
    req = session.get(url, headers={'User-Agent': ua.random})
    print(req)
    req.encoding = 'utf-8'
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    each_news = soup.find_all('div', {'class': 'listing-item__content'})
    if each_news:
        print(len(each_news))
    for one_piece in each_news:
        link = one_piece.find('a', {'class': 'listing-preview__content'}).attrs['href']
        news.append(link)

  0%|          | 0/4 [00:00<?, ?it/s]

<Response [200]>


 25%|██▌       | 1/4 [00:01<00:04,  1.66s/it]

30


 50%|█████     | 2/4 [00:02<00:02,  1.36s/it]

<Response [200]>
30


 75%|███████▌  | 3/4 [00:03<00:01,  1.17s/it]

<Response [200]>
30


100%|██████████| 4/4 [00:03<00:00,  1.08it/s]

<Response [200]>
30


In [7]:
print(len(news))

120


Глянем, наберется ли в каждой из этой сотни новостей нужное количество слов, заодно посмотрим, что там происходит

In [8]:
link = news[10]
req = session.get(link, headers={'User-Agent': ua.random})
req.encoding = 'utf-8'
req = req.text
req = BeautifulSoup(req, 'html.parser')
new_text = req.find('div', {'class': "article__body"}) #.text.lower()
#print(new_text)
new_text = new_text.find_all('p') #.text.lower()
print(new_text)

[<p><strong>Лже-медики.</strong> Облапошивание начинается со звонка строгим голосом из некой официальной организации (названо может быть все что угодно — от районной поликлиники и санэпидемстанции до Минздрава и ВОЗ). Жертву ставят в известность, что она предположительно контактировала с носителем COVID-19 и теперь ей надо срочно сдать тест. А поскольку «контактные» лица обязаны соблюдать самоизоляцию, медики осуществят забор материала для анализа на дому. Далее к абоненту, пока он не успел прийти в себя, оперативно прибывает бригада «медиков», берет образец слюны, 5000 руб оплаты с учетом выезда на дом — и исчезает навсегда.</p>, <p><strong>Лже-полицейские.</strong> По оценке экспертов, когда в дверь звонит человек в полицейской форме со словами, что вам выписан штраф «за нарушение противоковидной безопасности», ему открывает 80% граждан. В основном с целью прояснить ситуацию и доказать, что нарушения не было. Но «полицейский» уверенно настаивает на своем, показывает заранее заготовле

Что и ожидалось от раздела Проишествия. Но тут точно есть сто слов!

Пойдем используем собранный список ссылок и обкачаем сайт нормально

In [9]:
def get_full_news(link_list):
    texts = []
    titles = []
    for link in tqdm(link_list):
        news = []
        req = session.get(link, headers={'User-Agent': ua.random})
        req.encoding = 'utf-8'
        req = req.text
        req = BeautifulSoup(req, 'html.parser')
        title = req.find('h1', {'class': "article__title"})
        titles.append(title)
        new_text = req.find('div', {'class': "article__body"})
        new_text = new_text.find_all('p')
        for p in new_text:
            p = p.text
            news.append(p)
        texts.append(news)
    return titles, texts

In [10]:
titles, texts = get_full_news(news)
if len(texts)==len(titles):
    print('all clear!')

100%|██████████| 120/120 [00:50<00:00,  2.36it/s]

all clear!


Ячейка для предобработки. Не удивляйтесь, почему я тут токенизирую, а потом передумываю - так надо, такой датафрейм специфический

In [11]:
def texts_process(texts):
    new_texts = []
    for text in texts:
        new_text = ''
        for paragraph in text:
            if not paragraph.startswith('\n') and not paragraph.startswith('Читайте также:'):
                new_text = new_text + paragraph +' '
                
        new_text = word_tokenize(new_text)
        new_text = ' '.join(new_text)
        new_texts.append(new_text)
    return new_texts

К заголовкам у нас не будет особых требований, мы по ним просто ориентируемся, так что просто сделаем служебную скучную функцию с непонятными переменными, чтобы преобразовывать их из типа Тег в тип Строка

In [12]:
def title_process(texts):
    new_texts = []
    for text in texts:
        text = text.text
        new_texts.append(text)
    return new_texts

Обработаем данные и соберем в наш первый датафрейм

In [14]:
import nltk
nltk.download('punkt')
clear_texts = texts_process(texts)
clear_titles = title_process(titles)
df = pd.DataFrame(list(zip(clear_titles, clear_texts)), 
               columns =['titles', 'full_texts']) 
df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,titles,full_texts
0,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ..."
1,"Мать, бросившая детей на АЗС, объяснила свой п...",34-летняя жительница Одинцова ехала на своей м...
2,Аферисты похитили у семьи крымского священника...,"По данным издания , жертвой мошенничества стал..."
3,Банда начала зверски убивать кошек в Лобне,Страх и ужас своей расправой над беззащитными ...
4,Студентка-экскурсовод умерла внезапной смертью...,Погибшая Дарья в числе остальных слушателей уч...
...,...,...
115,"В Саратове ""друг семьи"" годами насиловал школь...","По информации ведомства , следственным отделом..."
116,Отец убитой историком Соколовым Ещенко ответил...,Вчера на заседании суда в Санкт-Петербурге Сок...
117,"В Москве женщину заставили мастурбировать, а п...",Москвичка до полуночи отдыхала в одном из клуб...
118,Появились шокирующие подробности убийства тран...,"По версии следствия , в день убийства 20-летни..."


Это здорово! Сохраним, на всякий случай

In [ ]:
df.to_csv('textbase_news_articles.csv')

А вот сейчас веселье и пойдет. Эта функция служит для того, чтобы разделить полные тексты по предложениям с созданием новой строки датафрейма, у которой будет идентичный заголовок

In [15]:
lst_col = 'full_texts' 

x = df.assign(**{lst_col:df[lst_col].str.split('.')})

my_dataframe = pd.DataFrame({col:np.repeat(x[col].values, x[lst_col].str.len()) for col in x.columns.difference([lst_col])}).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]

In [16]:
my_dataframe

,titles,full_texts
0,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ..."
1,Жуткие подробности подготовки бойни в аграрном...,У него не складывались отношения со сверстник...
2,Жуткие подробности подготовки бойни в аграрном...,Это и послужило мотивом для подготовки массов...
3,Жуткие подробности подготовки бойни в аграрном...,Банин не хотел устраивать теракт в одиночку
4,Жуткие подробности подготовки бойни в аграрном...,Осенью 2019 года он начал активно переписыват...
...,...,...
2457,Обвиненный в изнасиловании дочери добился комп...,Лишь стараниями следователей СК он вновь обре...
2458,Обвиненный в изнасиловании дочери добился комп...,Однако об аресте стало известно друзьям и кол...
2459,Обвиненный в изнасиловании дочери добился комп...,"Чтобы восстановить свое доброе имя , отец пос..."
2460,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...


Она работает! То же самое, только делим предложения на слова с созданием новых строк - но с теми же заголовками и предложениями

In [17]:
my_dataframe['words'] = my_dataframe['full_texts']
lst_col = 'words' 

x = my_dataframe.assign(**{lst_col:my_dataframe[lst_col].str.split(' ')})

my_final_dataframe = pd.DataFrame({col:np.repeat(x[col].values, x[lst_col].str.len()) for col in x.columns.difference([lst_col])}).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]

In [18]:
my_final_dataframe

,titles,full_texts,words
0,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",Как
1,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",стало
2,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",известно
3,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",«
4,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",МК
...,...,...,...
42433,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,200
42434,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,тысяч
42435,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,рублей
42436,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,


Чудно. Стаковерфлоу творит чудеса. Скормим эти чудеса пайморфи и создадим для важной нам выдачи еще три столбца

In [19]:
my_final_dataframe['lemma'] = my_final_dataframe.apply(lambda row: morph.parse(row.words)[0].normal_form, axis = 1)
my_final_dataframe['form'] = my_final_dataframe.apply(lambda row: morph.parse(row.words)[0].tag, axis = 1)
my_final_dataframe['POS'] = my_final_dataframe.apply(lambda row: morph.parse(row.words)[0].tag.POS, axis = 1)

In [21]:
my_final_dataframe

,titles,full_texts,words,lemma,form,POS
0,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",Как,как,CONJ,CONJ
1,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",стало,стать,"VERB,perf,intr neut,sing,past,indc",VERB
2,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",известно,известно,"PRED,pres",PRED
3,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",«,«,PNCT,None
4,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",МК,мк,"NOUN,inan,masc,Sgtm,Fixd,Abbr,Orgn sing,nomn",NOUN
...,...,...,...,...,...,...
42433,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,200,200,"NUMB,intg",None
42434,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,тысяч,тысяча,"NOUN,inan,femn plur,gent",NOUN
42435,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,рублей,рубль,"NOUN,inan,masc plur,gent",NOUN
42436,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,,,UNKN,None


Казалось бы, все хорошо, но как же - у нас прорехи в датафрейме, помешают рядом стоящие слова замечать. Исправим.

In [22]:
my_final_dataframe.drop(my_final_dataframe[my_final_dataframe.words == ''].index, inplace=True)

In [23]:
my_final_dataframe

,titles,full_texts,words,lemma,form,POS
0,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",Как,как,CONJ,CONJ
1,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",стало,стать,"VERB,perf,intr neut,sing,past,indc",VERB
2,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",известно,известно,"PRED,pres",PRED
3,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",«,«,PNCT,None
4,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",МК,мк,"NOUN,inan,masc,Sgtm,Fixd,Abbr,Orgn sing,nomn",NOUN
...,...,...,...,...,...,...
42431,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,госказны,госказна,"NOUN,inan,femn sing,gent",NOUN
42432,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,–,–,PNCT,None
42433,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,200,200,"NUMB,intg",None
42434,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,тысяч,тысяча,"NOUN,inan,femn plur,gent",NOUN


И все бы ладно, да знаки препинания!

In [24]:
punkt = [',', ';', ':', '"']
my_final_dataframe.drop(my_final_dataframe[my_final_dataframe.words.isin (punkt)].index, inplace=True)
my_final_dataframe

,titles,full_texts,words,lemma,form,POS
0,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",Как,как,CONJ,CONJ
1,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",стало,стать,"VERB,perf,intr neut,sing,past,indc",VERB
2,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",известно,известно,"PRED,pres",PRED
3,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",«,«,PNCT,None
4,Жуткие подробности подготовки бойни в аграрном...,"Как стало известно « МК » , устроить « шутинг ...",МК,мк,"NOUN,inan,masc,Sgtm,Fixd,Abbr,Orgn sing,nomn",NOUN
...,...,...,...,...,...,...
42431,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,госказны,госказна,"NOUN,inan,femn sing,gent",NOUN
42432,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,–,–,PNCT,None
42433,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,200,200,"NUMB,intg",None
42434,Обвиненный в изнасиловании дочери добился комп...,Каширский суд постановил выплатить ему компен...,тысяч,тысяча,"NOUN,inan,femn plur,gent",NOUN


Чистота, красота, и только мертвые с косами стоят...мертвые индексы, разумеется. Которые уже совершенно не соответствуют настоящим и будут бесполезны при проверке соседства. Создадим-ка новые

In [25]:
my_final_dataframe['ind'] = [i for i in range(34748)]

In [26]:
my_final_dataframe[900:920]

,titles,full_texts,words,lemma,form,POS,ind
1109,Студентка-экскурсовод умерла внезапной смертью...,Дарья с широко открытыми глазами лежала на ас...,лежала,лежать,"VERB,impf,intr femn,sing,past,indc",VERB,900
1110,Студентка-экскурсовод умерла внезапной смертью...,Дарья с широко открытыми глазами лежала на ас...,на,на,PREP,PREP,901
1111,Студентка-экскурсовод умерла внезапной смертью...,Дарья с широко открытыми глазами лежала на ас...,асфальте,асфальт,"NOUN,inan,masc sing,loct",NOUN,902
1114,Студентка-экскурсовод умерла внезапной смертью...,"« Скорая помощь » , прибывшая спустя считанны...",«,«,PNCT,None,903
1115,Студентка-экскурсовод умерла внезапной смертью...,"« Скорая помощь » , прибывшая спустя считанны...",Скорая,скорый,"ADJF,Qual femn,sing,nomn",ADJF,904
1116,Студентка-экскурсовод умерла внезапной смертью...,"« Скорая помощь » , прибывшая спустя считанны...",помощь,помощь,"NOUN,inan,femn sing,accs",NOUN,905
1117,Студентка-экскурсовод умерла внезапной смертью...,"« Скорая помощь » , прибывшая спустя считанны...",»,»,PNCT,None,906
1119,Студентка-экскурсовод умерла внезапной смертью...,"« Скорая помощь » , прибывшая спустя считанны...",прибывшая,прибыть,"PRTF,perf,intr,past,actv femn,sing,nomn",PRTF,907
1120,Студентка-экскурсовод умерла внезапной смертью...,"« Скорая помощь » , прибывшая спустя считанны...",спустя,спустя,PREP,PREP,908
1121,Студентка-экскурсовод умерла внезапной смертью...,"« Скорая помощь » , прибывшая спустя считанны...",считанные,считать,"PRTF,perf,tran,past,pssv,Adjx plur,nomn",PRTF,909


Вуаля! Теперь можно с чистой совестью сохранять в базу данных. Ссв, правда. 

In [ ]:
my_final_dataframe.to_csv('clean_textbase_parsed.csv')

## MAIN

Итак, вот и пришло время для воплощения самой функции мэйн, которая будет искать в нашем корпусе нужные нам слова

Для начала, дернем из документации пайморфи список поддерживаемых тегов:

NOUN	имя существительное

ADJF	имя прилагательное (полное)

ADJS	имя прилагательное (краткое)

COMP	компаратив

VERB	глагол (личная форма)

INFN	глагол (инфинитив)

PRTF	причастие (полное)

PRTS	причастие (краткое)

GRND	деепричастие

NUMR	числительное

ADVB	наречие

NPRO	местоимение-существительное

PRED	предикатив

PREP	предлог

CONJ	союз

PRCL	частица

INTJ	междометие

Пригодится. Теперь осталась только одна маленькая декоративная деталь - класс ошибок, который поможет нам тихо-мирно выходить из ячейки, если пользователь косячит:

In [27]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

raise StopExecution

StopExecution: ignored

Его основные преимущества это, во-первых, то, что он не убивает кернел - а это довольно раздражающая штука, - а во-вторых, он тихо выходит и не сообщает пользователю, где там что из-за его головотяпства упало.

Итак, функции-обработчики! Им будут поступать запросы, для которых уже определено, чего от нас хотят - это мельчайшие функции на самом низком уровне пищевой цепи, и задачи у них довольно простые и связаны с непосредственным поиском по базе

In [28]:
allowed_tags = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']

def get_morphology(dataframe, word):
    #inputs dataframe and str, outputs dataframe and list
    #looks for the same word
    word1 = word.strip('"')
    df_word = dataframe.loc[dataframe['words'] == word1]
    sentences = df_word['full_texts'].tolist()
    if len(sentences)==0:
        print('К сожалению, такого слова нет в моем словаре!')
        raise StopExecution
    else:
        return df_word, sentences
    
def get_tagged(dataframe, word, allowed_tags=['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']
):
    #inputs dataframe, list and str, outputs dataframe and list
    #looks for specified lemma with specified POS 
    compl_word = word.split('+')
    tag = set(allowed_tags)&set(compl_word)
    if not tag:
        print('К сожалению, я не знаю этот тег!')
        raise StopExecution
    else:
        current_word = morph.parse(compl_word[0])[0]
        current_word_lemma = current_word.normal_form
        current_word_POS = current_word.tag.POS
        df_word_POS = dataframe.loc[(dataframe['lemma'] == current_word_lemma) & (dataframe['POS'] == current_word_POS)]
        sentences = df_word_POS['full_texts'].tolist()
        return df_word_POS, sentences
    
def get_lemma(dataframe, word):
    #inputs dataframe and str, outputs dataframe and list
    #looks for the same lemma
    word_lem_parsed = morph.parse(word)[0]
    word_lem_lemma = word_lem_parsed.normal_form
    df_lemma = dataframe.loc[(dataframe['lemma'] == word_lem_lemma)]
    sentences = df_lemma['full_texts'].tolist()
    if len(sentences)==0:
        print('К сожалению, такого слова нет в моем словаре!')
        raise StopExecution
    else:
        return df_lemma, sentences
    
def get_pos(dataframe, word):
    #inputs dataframe and str, outputs dataframe and list
    #looks for the same POS
    df_pos = dataframe.loc[(dataframe['POS'] == word)]
    sentences = df_pos['full_texts'].tolist()
    if len(sentences)==0:
        print('К сожалению, слов этой части речи нет в моем словаре!')
        raise StopExecution
    else:
        return df_pos, sentences
    

И вот, момент настал. Две самые крупные рыбы в этом море. Функция работы со словами - та самая станция сортировки, которая определяет, какому именно поисковику перенаправить запрос, и заодно отвечает пользователю, что именно он попросил. Функция работы с запросами токенизирует их по пробелам (зачем множить токенизаторы без необходимости?), и в зависимости от количества токенов выдает ответ: предложения, в которых встретились слова, и заголовки статей, откуда эти предложения к нам попали. Оптимизируем поиск мы постепенным сужением круга поиска - результат поиска по первому слову сужает круг, второе слово ищется уже в выдаче предложений для первого, третье - в выдаче для второго. 

In [29]:
def work_with_words(dataframe, word, allowed_tags=allowed_tags):
    trick = word[0]
    if trick=='"':
        print('вы запрашиваете слова этой леммы в этой форме')
        return get_morphology(dataframe, word)
    elif '+' in word:
        print('вы запрашиваете лемму с конкретной частью речи')
        return get_tagged(dataframe, word, allowed_tags=allowed_tags)
    elif word in allowed_tags:
        print('вы запрашиваете все слова по тегу этой части речи')
        return get_pos(dataframe, word)
    else:
        print('вы запрашиваете поиск по лемме')
        return get_lemma(dataframe, word)
    
def work_with_quiery(quiery):
    quiery_parsed = quiery.split(' ')
    print(quiery_parsed)
    if len(quiery_parsed)==1:
        print('1')
        answer,_ = work_with_words(my_final_dataframe, quiery_parsed[0], allowed_tags=allowed_tags)
        list_words = []
        for index, row in answer.iterrows():
            answer_word = str('Это слово встретилось в предложении:'+ row['full_texts'] + '. | которое было в статье: ' + row['titles'])
            list_words.append(answer_word)
        return list_words
    
    elif len(quiery_parsed)==2:
        print('2')
        list_sentences = []
        first_answer, first_sentences = work_with_words(my_final_dataframe, quiery_parsed[0], allowed_tags=allowed_tags)
        indexes = first_answer['ind'].tolist()
        indexes = [x + 1 for x in indexes]
        second_answer_pool = my_final_dataframe.loc[my_final_dataframe['full_texts'].isin(first_sentences)]
        second_answer, second_sentences = work_with_words(second_answer_pool, quiery_parsed[1], allowed_tags=allowed_tags)
        second_indexes = second_answer['ind'].tolist()
        right_index = set(indexes)&set(second_indexes)
        if not right_index:
            print('К сожалению, в моем корпусе нет ничего по вашему запросу')
        else:
            for elem in right_index:
                    locate_row = my_final_dataframe.loc[my_final_dataframe['ind'] == elem]
                    for index, row in locate_row.iterrows():
                        answer_word = str('Эти слова встретилось в предложении:'+ row['full_texts'] + '. | которое было в статье: ' + row['titles'])
                        list_sentences.append(answer_word)
        return list_sentences
    
    elif len(quiery_parsed)==3:
        print('3')
        list_sentences = []
        first_answer, first_sentences = work_with_words(my_final_dataframe, quiery_parsed[0], allowed_tags=allowed_tags)
        indexes = first_answer['ind'].tolist()
        indexes = [x + 1 for x in indexes]
        second_answer_pool = my_final_dataframe.loc[my_final_dataframe['full_texts'].isin(first_sentences)]
        second_answer, second_sentences = work_with_words(second_answer_pool, quiery_parsed[1], allowed_tags=allowed_tags)
        second_indexes = second_answer['ind'].tolist()
        right_index = set(indexes)&set(second_indexes)
        if not right_index:
            print('К сожалению, в моем корпусе нет ничего по вашему запросу')
        else:
            right_index = [x + 1 for x in right_index]
            third_answer_pool = my_final_dataframe.loc[my_final_dataframe['full_texts'].isin(second_sentences)]
            third_answer, third_sentences = work_with_words(third_answer_pool, quiery_parsed[2], allowed_tags=allowed_tags)
            third_indexes = third_answer['ind'].tolist()
            master_index = set(right_index)&set(third_indexes)
            if not master_index:
                print('К сожалению, в моем корпусе нет ничего по вашему запросу')
            else:
                for elem in master_index:
                    locate_row = my_final_dataframe.loc[my_final_dataframe['ind'] == elem]
                    for index, row in locate_row.iterrows():
                        answer_word = str('Эти слова встретились в предложении:'+ row['full_texts'] + ', которое было в статье: ' + row['titles'])
                        list_sentences.append(answer_word)
        return list_sentences        
    else:
        print ('К сожалению, я не обрабатываю запросы длиннее трех токенов')
        raise StopExecution

Тадаам! Вот и мэйн. У него здесь дел немного - еще раз проверить, не возникла ли нигде ошибка с ненайденными ответами, которую мы не перехватили, а пользователю видеть и не надо, и предложить пользователю дать запрос. 

In [30]:
def main():
    quiery = input('Что ищем? Не более трех слов в запросе! ')
    try:
        answer = work_with_quiery(quiery)
        for i in answer:
            print(i)
            print('\n')
    except TypeError:
        print('Что-то пошло не так. Попробуйте переформулировать запрос')
        raise StopExecution

In [31]:
main()

Что ищем? Не более трех слов в запросе! дом+NOUN
['дом+NOUN']
1
вы запрашиваете лемму с конкретной частью речи
Это слово встретилось в предложении: - Они сказали , что видели троих подростков , которые ходили с палками , просовывали их в продухи в фундаменте домов и проверяли , не греются ли в подвалах кошки . | которое было в статье: Банда начала зверски убивать кошек в Лобне


Это слово встретилось в предложении: В интернете они познакомились с Некрасовым и под предлогом вступления в интимные отношения договорились о встрече дома у Некрасова . | которое было в статье: Российский силовик замучил до смерти помощника прокурора из-за сексуальной ориентации


Это слово встретилось в предложении: - Женщина открыла свое ООО по пошиву одежды , и , пока трудилась на работе , ее дочка все делала по хозяйству дома . | которое было в статье: Девочка сбежала от матери, делавшей из нее рабыню


Это слово встретилось в предложении: А поскольку « контактные » лица обязаны соблюдать самоизоляцию , ме